In [1]:
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import collections
from pprint import pprint
from glob import glob

In [39]:
dir_names = ['./mae_data/val/*', './mae_data/train/*']

json_arr = []

for name in dir_names:
    json_arr += glob(name)
    
len(json_arr)

900

In [41]:
df_arr = []

for jj in tqdm(json_arr):
    df_t = pd.read_json(jj)
    df_arr.append(df_t)
    
df = pd.concat(df_arr, ignore_index=True)

In [7]:
df.text[7]

'Diesel newcomer Arges takes a turn for the sophisticated, incorporating a rich brown leather strap, a sleek stainless steel case and cool grey dial.\nCase Size: 46 X 50 mm\nCase Thickness: 12 mm\nLug Width: 24 mm\nWater Resistance: 10 ATM'

In [9]:
img_label = pd.read_csv('./mae_data/csv/mae_img_label.csv', header=None)

In [10]:
img_label.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,product%7C4%7C-1000000248_0.jpg,JPEG,RGB,"(335, 500)",40.0 KB,bow_tie,99.928397,web_site,0.022756,Windsor_tie,0.016022,book_jacket,0.008080,2912,277
1,product%7C4%7C-1000000416_0.jpg,JPEG,RGB,"(182, 300)",12.7 KB,packet,75.721252,web_site,7.337227,lipstick,5.100508,face_powder,2.705671,2912,277
2,product%7C4%7C-1000002752_0.jpg,JPEG,RGB,"(308, 308)",7.6 KB,rule,78.719521,slide_rule,1.440430,hair_spray,1.379705,web_site,1.271158,2912,277
3,product%7C4%7C-1000004848_0.jpg,JPEG,RGB,"(315, 315)",9.3 KB,web_site,36.242899,rule,22.350354,broom,3.967119,bow,3.919487,2912,277
4,product%7C4%7C-1000006328_0.jpg,JPEG,RGB,"(400, 300)",4.9 KB,plunger,17.542212,candle,10.454264,beaker,6.337325,hourglass,6.316449,2912,277
5,product%7C4%7C-1000010932_0.jpg,JPEG,RGB,"(180, 180)",2.8 KB,lotion,81.982458,sunscreen,8.297831,soap_dispenser,4.743769,nipple,2.669959,2912,277
6,product%7C4%7C-1000011740_0.jpg,JPEG,RGB,"(275, 275)",19.0 KB,loupe,83.716655,face_powder,3.919664,guillotine,3.425708,printer,1.406303,2912,277
7,product%7C4%7C-1000016271_0.jpg,JPEG,RGB,"(300, 40)",3.2 KB,dumbbell,26.946607,screw,10.507034,spotlight,7.382729,binoculars,4.871642,2912,277
8,product%7C4%7C-100002160_0.jpg,JPEG,RGB,"(331, 499)",55.9 KB,book_jacket,95.286155,menu,1.590985,comic_book,0.478301,Dungeness_crab,0.419812,2912,277
9,product%7C4%7C-1000021724_0.jpg,JPEG,RGB,"(2000, 2000)",728.8 KB,spider_web,69.914871,fountain,19.313847,balloon,1.809467,garden_spider,1.002125,2912,277


In [11]:
output_df = pd.DataFrame(columns=['title', 'images'])

In [30]:
pbar = tqdm(total=len(img_label))

for idx, row in img_label.iterrows():    
    pbar.update(1)
    
    img_name = row[0].replace('%7C', '|')[:-4]
    class_labels = [row[5], row[7], row[9], row[11]]
    mask = df.images.apply(lambda x: img_name in x)
    df1 = df[mask]
    
    if len(df1['title']) == 0:
        continue
    
    for i, x in df1.iterrows():
        output_df = output_df.append({'title': x['title'], 'images': class_labels}, ignore_index=True)

In [14]:
t = output_df.iloc[1]['title']
labels = output_df.iloc[1]['images']

In [27]:
print('title ', t)
print('label ', labels)

title  cubic
label  ['refrigerator', 'plate_rack', 'rotisserie', 'stove']


In [23]:
from gensim.models import Word2Vec
model = Word2Vec.load('./mae_data/gensim_model/apnews_sg/word2vec.bin')
vocab = list(model.wv.vocab)
model['core'].shape

C:\Users\LikeJust\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(300,)

In [24]:
titles = [x.lower() for x in t.split()]
label_arr = []

for l in labels:
    if '_' in l:
        for ll in l.split('_'):
            label_arr.append(ll)
    else:
        label_arr.append(l)
        
titles = list(filter(lambda x: x in vocab, titles))
label_arr = list(filter(lambda x: x in vocab, label_arr))

In [25]:
print(label_arr)
print(titles)

['refrigerator', 'plate', 'rack', 'rotisserie', 'stove']
['danby', 'silhouette', 'built-in', 'beverage', '5.6', 'cubic']


In [26]:
for l in label_arr:
    for t in titles:
        sim = model.wv.similarity(l, t)
        if sim < 0.05:
            print(l, t, '=', sim)

refrigerator 5.6 = 0.012576495
rack danby = 0.03602119
rack cubic = 0.03701368
rotisserie 5.6 = 0.03124197
